In [38]:
import re
from os import listdir
from os.path import isfile, join
import random

In [39]:
def split_script(src, dest, block_count, test_count):
    with open(src, "r") as f:
        lines = f.readlines()
        line_count = len(lines)
        lines_per_block = line_count // block_count
        blocks = random.sample(range(0, block_count), test_count)
        dests = []
        line_chunks = []
        for i in range(test_count):
            dests.append(dest[:-4] + "_" + str(i+1) + "_input.txt")
            dests.append(dest[:-4] + "_" + str(i+1) + "_true_output.txt")
            
            start_index = blocks[i]*lines_per_block
            end_index = start_index + lines_per_block
            split_index = (start_index + end_index) // 2
            
            line_chunks.append(lines[start_index:split_index])
            line_chunks.append(lines[split_index+1:end_index])
        
        for n in range(len(dests)):
            with open(dests[n], "w+") as f:
                f.write(''.join(line_chunks[n]))
            

def generate_tests_entire_dir(srcpath, destpath):
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            split_script(src, dest, 20, 3)
    
def parse_script(src, dest):
    with open(src, "r") as f:
        parsed = ""
        added_new = True # Used for ignoring consequtive empty lines
        for line in f:
            temp = line.strip(" ")
            if temp != "\n":
                temp = temp.strip("\n")
                if (re.search('[a-zA-Z]', temp)): # Ignore page number lines
                    parsed += line.strip() + " "
                    added_new = False
            else:
                if not added_new:
                    parsed += "\n"
                    added_new = True
                continue
    with open(dest, "w") as f:
        f.write(parsed)

def process_entire_dir(srcpath, destpath):
    for f in listdir(srcpath):
        src = join(srcpath, f)
        dest = join(destpath, f)
        if isfile(src):
            parse_script(src, dest)
    print("Processed: {} scripts".format(len(listdir(srcpath))))

In [ ]:
process_entire_dir("./raw_data/", "./parsed_data/")

In [40]:
generate_tests_entire_dir("./parsed_data/", "./test_cases/")